<a href="https://colab.research.google.com/github/aytnihca/Sentiment_Analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from textblob import TextBlob
from collections import Counter
import warnings

In [3]:
df_train = pd.read_csv('drugsComTrain_raw.csv')
df_train

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,19-Sep-15,79


In [4]:
df_train.isnull().sum()
df_train = df_train.dropna()
df_train

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,31-May-15,125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,1-Nov-11,34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,15-Mar-14,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,19-Sep-15,79


In [5]:
top_10_conditions = ['ADHD', 'Birth Control', 'Depression', 'Bipolar Disorde', 'Obesity', 'Insomnia', 'Acne', 'Anxiety', 'Pain', 'Weight Loss']

In [6]:
df_train = df_train[df_train['condition'].isin(top_10_conditions)]
df_train = df_train.head(10000)
df_train

,uniqueID,drugName,condition,review,rating,date,usefulCount
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,14-Mar-15,32
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1
...,...,...,...,...,...,...,...
21763,42838,Xulane,Birth Control,"""This is the first review I&#039;ve ever wrote...",1,15-Sep-16,12
21773,128992,Levonorgestrel,Birth Control,"""I got Skylah and it was horribly painful. Wor...",9,30-Nov-15,2
21774,98694,Nexplanon,Birth Control,"""I have had the Nexplanon in for 9 months . Ge...",7,22-Dec-13,4
21775,8160,Zolpidem,Insomnia,"""After getting it and reading on the Internet ...",10,19-Nov-16,72


In [8]:
df_train_subset = df_train.sample(n=3000, random_state=1)

In [9]:
#Using the `drop()` method to remove the old index column after resetting the index
df_train_subset = df_train_subset.reset_index().drop('index', axis=1)

In [10]:
lst = df_train_subset['review'].tolist()
matched_list = [ele for ele in lst if '&#039' in ele]
print('number of rows with "&#039" character: ', len(matched_list))

number of rows with "&#039" character:  2071


In [11]:
#converting the junk keyword
df_train_subset['review'] = pd.Series(df_train_subset['review']).replace('&#039;', "'", regex=True)

In [12]:
contractions_mapping  = {
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
contractions_pattern = re.compile('(%s)' % '|'.join(contractions_mapping.keys()))

def expand_contractions_func(text, contractions_map=contractions_mapping):
    def replace_contractions(match):
        return contractions_map[match.group(0)]

    return contractions_pattern.sub(replace_contractions, text)

df_train_subset['cleaned_review'] = df_train_subset['review'].apply(expand_contractions_func)

In [13]:
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
import spacy
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
exclude_list = ['but', 'no', 'because', 'against', 'not', 'than']
stop_words = set(stopwords.words('english'))
stop_words.difference_update(exclude_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
count = 0
def normalize_punc_lemmatize(text):
    global count
    count = count + 1
    #normalizing the text by lowering to same case
    text = text.lower()
    #removing the punctuations
    text_p = "".join([char for char in text if char not in string.punctuation])
    #lemmatizing the text
    doc = nlp(text_p)
    sen = " ".join([token.lemma_ if token.lemma_ != '-PRON-' else str(token) for token in doc])
    #print(count)
    return sen

df_train_subset['cleaned_review'] = df_train_subset['cleaned_review'].apply(normalize_punc_lemmatize)

In [15]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [16]:
import torch
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Move the model to the GPU
model.to(device)

def sentiment_analysis(text, tokenizer, model):
    MAX_LEN = 512
    inputs = tokenizer(text, return_tensors="pt", max_length=MAX_LEN, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the GPU
    outputs = model(**inputs)
    logits = outputs.logits
    probs = softmax(logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()

    return predicted_class, probs.cpu().detach().numpy()

def process_batch(data, batch_size):
    batch_start = 0
    batch_end = batch_size
    sentiments = []

    for _ in tqdm(range(0, len(data), batch_size)):
        batch_data = data.iloc[batch_start:batch_end]
        batch_sentiments = batch_data.apply(lambda x: sentiment_analysis(x, tokenizer, model))
        sentiments.extend(batch_sentiments)

        batch_start += batch_size
        batch_end += batch_size

    return sentiments

batch_size = 50
df_train_subset['sentiment'] = process_batch(df_train_subset['cleaned_review'], batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

100%|██████████| 60/60 [10:07<00:00, 10.12s/it]
